In [ ]:
%pip install pandas matplotlib scipy seaborn

In [ ]:
import os
import argparse
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
import seaborn as sns
import glob
from scipy import stats
from dateutil import tz

In [ ]:
from common import combine_csvs

In [ ]:
MEGAb_TO_b = 1e6
TCP_DOWN = "*down.*.csv"
UDP_DOWN = "*down*udp*.csv"
TCP_UP = "*receive.*[!p].csv"
UDP_UP = "*receive*.udp.csv"
TEMP='Temp (°C)'
PRECIP='Precip. Amount (mm)'

JITTER = 'jitter_ms'
BANDWIDTH = 'bandwidth'

In [ ]:
#matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
sns.set_context("paper", rc={"font.size":9,"axes.titlesize":9,"axes.labelsize":8, "xtick.labelsize":8})

In [ ]:
def convert_to_mb(df):
    # Converts to Megabits per second
    df.bits_per_second /= MEGAb_TO_b
    df.rename(columns={'bits_per_second': 'bandwidth'}, inplace=True)

In [ ]:
def concat_df(src, pattern, keep=['bandwidth']):
    combined_df = combine_csvs(glob.glob(f"{src}/**/{pattern}", recursive=True))
    
    combined_df.index = pd.to_datetime(combined_df.index, unit='s')
    convert_to_mb(combined_df)
    combined_df = combined_df[keep]
    return combined_df

In [ ]:
def plot_time(args):
    from scipy.signal import savgol_filter

    #df = concat_df(args.src_folder, UDP_UP, [JITTER])
    df = combine_reg(args.src_folder, UDP_UP, first=4)
    df_unstacked = df.unstack().dropna()
    df_unstacked = df_unstacked.mask(df_unstacked == -1).reset_index(name=BANDWIDTH).set_index('timestamp')
    df_regs = df.replace(-1, pd.NA)
    df_regs = df_regs.dropna(how='all').astype(float)
    #df = concat_df(args.src_folder, UDP_DOWN).sort_values('timestamp')
    df_regs = df_regs.loc['2022-05-28 18:13:00':]
    #df = df.loc['2022-03-01 04:07:30':'2022-03-01 04:08:40']
    #df = df.loc['2022-03-01':'2022-03-02']

    print(df_regs)

    # Aggreagation options
    df_regs_interp = df_regs.resample('2T').mean()
    df_regs_interp = df_regs_interp.interpolate(method='linear')
    df_regs_interp = df_regs_interp.apply(lambda x: savgol_filter(x,41,1))
    # Perform savgol filtering
    print(df_regs.head())

    #df.to_csv("out.csv", encoding='utf-8-sig')

    print(df_unstacked.shape[0])

    #fig, ax = plt.subplots(figsize=(3.5,3))
    fig, ax = plt.subplots(figsize=(20,10))

    ax.xaxis.update_units(df.index)
    y_val = df_unstacked.bandwidth
    #sns.scatterplot(x=ax.xaxis.convert_units(df.timestamp), y=y_val, ax=ax)
    #sns.jointplot(x=ax.xaxis.convert_units(df.timestamp), y=y_val, ax=ax)

    #sns.lineplot(x=ax.xaxis.convert_units(df.index), y=y_val, ax=ax, hue=y_val.isna().cumsum(),
    #        palette=["black"]*sum(y_val.isna()), markers=True, legend=False)
    #sns.lineplot(data=df_regs)
    sns.lineplot(data=df_regs_interp)
    #ax.set(yscale='log')
    for label in ax.get_xticklabels():
        label.set_rotation(45)
        label.set_ha('right')

    #ax.set_ylabel("Jitter (ms)")
    ax.set_ylabel("Bandwidth (Mb/s)")
    if args.name:
        ax.set_title(args.name)

    fig2, ax2 = plt.subplots(figsize=(20,10))
    sns.ecdfplot(data=df_regs)

    plt.tight_layout()
    plt.savefig(f'{args.filename}.eps', format='eps', bbox_inches='tight')
    plt.show()